# MiniProject

## 프로젝트 설명

무신사는 대형 온라인 쇼핑몰이다. 자주 이용하고 있지만 평소에 랭킹 첫 페이지에 있는 제품 중
판매량, 조회수가 저조하거나 생소한 브랜드의 제품들이 top 100안에 있는 것을 보고 왜 이런 제품이 있는지
의문이 들었었다. 다른 사람들도 이런 점을 느끼고 무신사측에 광고를 해주려고 이런 제품들을 상위 랭크에 넣어준것이냐
물어봤지만 돌아오는 답변은 어떠한 조작도 없다고 했다.

그래서 제일 많이 구매하는 상의 카테고리의 랭킹 첫 페이지의 데이터들을 모아서 랭킹에 영향을 끼치는 조회수, 판매량,
좋아요를 통해 평균에 많이 못 미치는 의심이 가는 제품들을 찾아보고자 이 프로젝트를 진행했다.

In [3]:
import time
import numpy as np
from collections import Counter
import warnings
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import re
import matplotlib
from matplotlib import font_manager, rc
import platform
if platform.system() == "Windows":
    font_name = font_manager.FontProperties(
        fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
matplotlib.rcParams['axes.unicode_minus'] = False

warnings.filterwarnings("ignore")

# 초기 페이지 넘버
page_num = 1

# 크롬 드라이버 실행
driver = webdriver.Chrome("chromedriver.exe")

# url로 이동
url = "https://search.musinsa.com/ranking/best?period=now&age=ALL&mainCategory=001&subCategory=&leafCategory=&price=&golf=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page={}&viewType=small&device=&priceMin=&priceMax=".format(page_num)
driver.get(url)
time.sleep(3)

# 현재 페이지 파싱
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")



In [121]:
# 상세 페이지 이동 함수


def clickProduct(page_num):
    next_page = driver.find_elements_by_css_selector("#goodsRankList > li:nth-child({}) > div.li_inner > div.list_img > a > \
    img".format(page_num))[0]
    next_page.click()
    time.sleep(2)


# 데이터 추출 함수
data = []


def getContents(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # brand name
    brand = soup.select("strong > a")[0].text

    # 제품명
    product = soup.select(
        ".right_contents.section_product_summary > span > em")[0].text

    # 가격
    price = soup.select("#list_price")[0].text

    # 좋아요
    like = soup.select(
        "#product-top-like > p > span.prd_like_cnt")[0].text

    # 평점
    star = soup.select(".prd-score__rating")

    if star:
        star = soup.select(".prd-score__rating")[0].text
    else:
        star = "0"

    # 한 달간 조회수
    view = soup.select("#pageview_1m")[0].text
    if view:
        view_re = re.sub("[이상]", "", view)
        view = view_re.strip()
    else:
        view = "0"
    # 판매 수
    sale = soup.select("#sales_1y_qty")[0].text
    if sale:
        sale_re = re.sub("[이상]", "", sale)
        sale = sale_re.strip()
    else:
        sale = "0"

    data.append([brand, product, price, like, star, view, sale])
    return data

In [ ]:
# 데이터 추출

for page_num in range(1,91):
    try:
        clickProduct(page_num)
        time.sleep(2)
        data = getContents(driver)
        driver.back()
        time.sleep(2)
#         print("%s 까지 완료"%page_num)
    except:
        time.sleep(4)
#         print("여기서 걸리네 %s"%page_num+1)
        clickProduct(page_num)
        
# print(data)

In [ ]:
# rank 추가

rank = 0

for dt in data:
    rank +=1
    dt.insert(0,rank)

In [ ]:
# 멤버 등급별 할인 가격 범위 -> 정가

for i in range(len(data)):
    if not re.match("[\d,]+ ~ ",data[i][3]):
        pass
    else:
        data[i][3] = re.sub("[\d,]+ ~ ", "",data[i][3])

In [ ]:
# column 추가

df = pd.DataFrame(data)
df.columns  = ["Rank","Brand","Product", "Price", "Like", "Grade", "View", "Sale"]

In [ ]:
# 엑셀파일로 저장

df.to_excel("musinsa_top_ranking.xlsx",index = False)

In [5]:
# 저장된 파일 불러오기

raw_data = pd.read_excel("musinsa_top_ranking.xlsx")
raw_data

,Rank,Brand,Product,Price,Like,Grade,View,Sale
0,1,COVERNAT,쿨 코튼 2-PACK 티셔츠,"34,400원","71,007",4.8,49.4만 회,8.9만 개
1,2,MARK GONZALES,슈무 아치로고 반팔 티셔츠 화이트,"23,400원","39,529",4.8,24만 회,2.2만 개
2,3,LEE,빅 트위치 로고 티셔츠 화이트,"21,000원","76,376",4.8,36.6만 회,5.2만 개
3,4,YALE,EMBROIDERY HANDSOME DAN TEE WHITE,"27,300원","27,319",4.9,27.5만 회,2.5만 개
4,5,LAFUDGESTORE,시티보이 빅오버 옥스포드 반팔 셔츠_Sky Blue,"39,000원","28,584",4.8,47만 회,2.5만 개
...,...,...,...,...,...,...,...,...
94,95,BEANPOLE MEN,[SLIM] Unisex 베이지 리넨 솔리드 셔츠 (BC1364A07A),"132,500원",10,5.0,0,0
95,96,FRIZMWORKS,ROUNDING OVERSIZED SHIRT _ INDIAN PINK,"40,600원",96,5.0,1.5천 회,50개
96,97,LMC,LMC CANDLE BEAR TEE white,"27,300원","9,402",4.8,3.7만 회,2.3천 개
97,98,CALVIN KLEIN JEANS,[CK] 남 J318111 YAF 화이트 슬림핏 모노그램 로고 폴로 반팔 티셔츠,"76,100원",7,0.0,0,0


In [6]:
# 컬럼 별 리스트로 저장
data = []
sale_data = list(raw_data["Sale"])
price_data = list(raw_data["Price"])
rank_data = list(raw_data["Rank"])
brand_data = list(raw_data["Brand"])
product_data = list(raw_data["Product"])
like_data = list(raw_data["Like"])
grade_data = list(raw_data["Grade"])
view_data = list(raw_data["View"])

# "개", "회" 제거 함수

def delete_hangeul(data):
    for i in range(len(data)):
        check = re.search("[ ]*[회개]",data[i])
        if check:
            data[i] = re.sub("[ ]*[회개]","",data[i])
            
# view, sale 한글 단위 제거

delete_hangeul(view_data)
delete_hangeul(sale_data)

# 한글로 된 단위 => 숫자로

def word_to_digit(data):
    for i in range(len(data)):
        if re.search("미만",data[i]):
            data[i] = re.sub("미만", "",data[i])
        elif re.search(",", data[i]):
            data[i] = re.sub(",","",data[i])

        elif re.search("천", data[i]):
            data[i] = re.sub("천","*1000",data[i])
        elif re.search("만", data[i]):
            data[i] = re.sub("만","*10000",data[i])

        data[i] = eval(data[i])
        data[i] = int(data[i])
        
# sale, view, like 한글로 된 단위 변경

word_to_digit(sale_data)
word_to_digit(view_data)
word_to_digit(like_data)

# data 리스트에 정보 담기

for i in range(len(sale_data)):
    data.append([rank_data[i],brand_data[i],product_data[i],price_data[i],like_data[i],grade_data[i],view_data[i],sale_data[i]])
    
# 데이터의 평균 구하는 함수

def avg(data):
    data_sum = 0
    avg = 0
    for i in data:
        data_sum += int(i)
    avg = data_sum / len(data)
    return int(avg)

avg_sale = avg(sale_data)
avg_view = avg(view_data)
avg_like = avg(like_data)

# 의심이 되는 제품, 카운트 넣을 딕셔너리 생성

doubt = {}

# 평균이랑 비교하여 의심되는 데이터 발견 시 추가하는 함수

def check_doubt_product(data,avg_data):
    for idx ,x in enumerate(data):
        if x < avg_data * 0.05:
#             print("{0}번째 데이터에서 이상 발견 (제품명 : {1})".format(idx,product_data[idx]))
#             print("평균 대비 {:.1f}%".format(x/avg_data*100))
            if doubt.get("%s"%product_data[idx]):
                doubt["%s"%product_data[idx]]+=1
            else:
                doubt["%s"%product_data[idx]] = 1

# 판매량, 조회수, 좋아요를 통해 저조한 제품 찾기
check_doubt_product(sale_data,avg_sale)
check_doubt_product(view_data,avg_view)
check_doubt_product(like_data,avg_like)

print("의심되는 제품 리스트")

for i in range(len(doubt.keys())):
    if list(doubt.values())[i] > 2:
        doubt_product = list(doubt.keys())[i]
        for i in data:
            if doubt_product in i:
                print("rank : ",i[0],"brand : ",i[1],"product : ",doubt_product)# view, sale 한글 단위 제거

의심되는 제품 리스트
rank :  19 brand :  MARK&LONA product :  M LOGO POINT SS POLO
rank :  32 brand :  A.P.C. product :  Arsenal T-Shirt
rank :  56 brand :  LACOSTE product :  남성 프렌치 폴로 PH731E-51N 031
rank :  57 brand :  POLO RALPH LAUREN product :  슬림핏 리넨 셔츠 - 블루
rank :  67 brand :  POLO RALPH LAUREN product :  슬림핏 리넨 셔츠 - 블루
rank :  59 brand :  LACOSTE product :  남성 프렌치 폴로 PH731E-51N 166
rank :  60 brand :  KODAK product :  클럽 폴로 니트 우먼 BLACK
rank :  62 brand :  ARCHIVE BOLD product :  [SET] 939 LOGO HOOD SET (DARK GRAY)
rank :  69 brand :  MLB product :  PRIDE TAG 스트라이프 반팔 셔츠 NY (SKY BLUE)
rank :  79 brand :  HIDEOUT product :  CHECK HALF SHIRTS (Light Khaki)
rank :  95 brand :  BEANPOLE MEN product :  [SLIM] Unisex 베이지 리넨 솔리드 셔츠 (BC1364A07A)
rank :  96 brand :  FRIZMWORKS product :  ROUNDING OVERSIZED SHIRT _ INDIAN PINK
rank :  98 brand :  CALVIN KLEIN JEANS product :  [CK] 남 J318111 YAF 화이트 슬림핏 모노그램 로고 폴로 반팔 티셔츠
rank :  99 brand :  13MONTH product :  STRIPE TURTLE NECK KNIT TOP (BROWN)
